In [1]:
import pandas as pd
import pyodbc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
import psycopg2 as pg
from urllib.parse import quote_plus
%matplotlib inline

In [2]:
# Database connections
username = input('Username for the rds ' )
password = input('Password to the rds ' )
server = input('Servername ' )
conrds = pyodbc.connect(driver = '{ODBC Driver 13 for SQL Server}',
                    server = str(server) +',1433',
                    database = 'water_crisis_proc',
                    user = str(username),
                    password = str(password))

conn =  'DRIVER={ODBC Driver 13 for SQL Server};gather-cpt-team-7.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com;DATABASE=water_crisis_proc;UID='+username+';PWD='+password+''
quoted = quote_plus(conn)
new_con = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_con)

In [26]:
consumption_df = pd.read_sql('select * from DemandConsumption', con = conrds)

population_df = pd.read_sql('select * from PopulationEstimate', con = conrds)

In [27]:
consumption_df.head()

,demand_consumption_id,urban,agri,date
0,1,564,0,2018-10-01
1,2,520,0,2018-09-01
2,3,540,0,2018-08-01
3,4,527,0,2018-07-01
4,5,556,0,2018-06-01


In [28]:
population_df.head()

,population_id,population1,population2,date
0,1,1567152,1581853,1980-01-01
1,2,1620588,1640692,1981-01-01
2,3,1675355,1701610,1982-01-01
3,4,1731434,1764436,1983-01-01
4,5,1788802,1829002,1984-01-01


In [29]:
consumption_df['combined'] = consumption_df['agri']+consumption_df['urban']

In [30]:
consumption_df.head()

,demand_consumption_id,urban,agri,date,combined
0,1,564,0,2018-10-01,564
1,2,520,0,2018-09-01,520
2,3,540,0,2018-08-01,540
3,4,527,0,2018-07-01,527
4,5,556,0,2018-06-01,556


In [37]:
def outliers(consumption_type):
    df = consumption_df
    df.index = df['date']
    
    above = df[consumption_type].mean() + 2*df[consumption_type].std()
    below = df[consumption_type].mean() - 2*df[consumption_type].std()
    
    
    df = df[consumption_type][(df[consumption_type]>above)|(df[consumption_type]< below)]
    
    return df

In [38]:
df = outliers('urban')

In [41]:
df

date
2018-09-01    520
2018-08-01    540
2018-07-01    527
Name: urban, dtype: int64